In [1]:
import pandas as pd
import numpy as np
import gc

from rec_lib.utils import reduce_mem_usage, set_target, customers_prep, articles_prep, df_lvl2_prep, top_12_recs
from rec_lib.utils import get_features, get_preds_result, col_convert_int
from rec_lib.models import get_lvl2_model_validation_preds, get_lvl2_model_validation_LGBM_preds
from rec_lib.metrics import precision_at_k, ap_k, recall, recall_at_k

import warnings
warnings.filterwarnings("ignore")

In [2]:
%load_ext autoreload
%autoreload 2

### Загрузка данных

In [3]:
%%time
result = pd.read_parquet('archive/validation_result.parquet')
customers = pd.read_parquet('archive/customers.parquet')
articles = pd.read_parquet('archive/articles.parquet')
transactions = pd.read_parquet('archive/transactions_train_for_power_bi.parquet')

CPU times: user 7.18 s, sys: 2.17 s, total: 9.35 s
Wall time: 4.55 s


### Подготовка данных

In [4]:
%%time
# преобразование датафрейма пользователей
customers = customers_prep(customers)

CPU times: user 1.68 s, sys: 188 ms, total: 1.86 s
Wall time: 1.85 s


In [5]:
%%time
# преобразование датафрейма категорий
articles = articles_prep(articles)

CPU times: user 31.4 ms, sys: 30.7 ms, total: 62.1 ms
Wall time: 60.8 ms


### Собираем датафрейм для обучения модели 2го уровня

In [6]:
%%time
# добавим нужные признаки 
result = df_lvl2_prep(result)

CPU times: user 1min 33s, sys: 8.96 s, total: 1min 42s
Wall time: 1min 41s


In [7]:
%%time
# добавляем данные из датафремов покупателей и категорий
result = result.merge(customers, on='customer_id', how='left')
result = result.merge(articles, on='article_id', how='left')

CPU times: user 23.5 s, sys: 9.26 s, total: 32.8 s
Wall time: 32.8 s


In [8]:
del customers;
del articles;
gc.collect()

0

### Добавляем фичи
Добавляем следующие фичи:
1. сумма продаж по категориям товаров
2. средняя сумма покупок по категориям товаров
3. количество заказов покупателя с товарами категории
4. количество единиц товаров категории, приобретенное покупателем
5. сумма покупок покупателя по категории
6. средний чек покупателя по категории
7. общая сумма покупок покупателя
8. средний чек покупателя

In [9]:
%%time
result = get_features(result, transactions)

CPU times: user 3min 43s, sys: 1min 57s, total: 5min 41s
Wall time: 5min 42s


#### Выбираем фичи для модели

In [10]:
# все используемые фичи
features = list(result)
features.remove('target')
features.remove('customer_id')
features.remove('article_id')

# категориальные фичи
cat_feats = features.copy()
cat_feats.remove('age')
cat_feats.remove('customer_id_short')
cat_feats.remove('article_id_short')

In [11]:
# убираем из категариальных фичей новые численные
cat_feats.remove('articles_buy_sum')
cat_feats.remove('articles_buy_avg')

cat_feats.remove('customers_articles_num_sum')
cat_feats.remove('customers_articles_num_count')

cat_feats.remove('customers_articles_sum')
cat_feats.remove('customers_articles_avg')

cat_feats.remove('customers_spent_sum')
cat_feats.remove('customers_spent_avg')

### Предсказания моделей CatBoost и их оценка

In [13]:
%%time
customers_id, article_id, target, preds = get_lvl2_model_validation_preds(result, features, cat_feats)

Получаем предсказания для возрастной группы 16-24
Получаем предсказания для возрастной группы 25-44
Получаем предсказания для возрастной группы 45-64
Получаем предсказания для возрастной группы 65-96
CPU times: user 4min 43s, sys: 48.3 s, total: 5min 32s
Wall time: 4min 53s


In [14]:
%%time
# построение датафрейма для получения оценок предсказаний модели
filename = 'validation_result'
result_test = get_preds_result(customers_id, article_id, target, preds, filename)

CPU times: user 29.2 s, sys: 755 ms, total: 29.9 s
Wall time: 29.5 s


In [15]:
print('Recall_own_rec_12 : ', result_test.apply(lambda row: recall(row['top_12_recs'], row['actual_article_id_short']), axis=1).mean())

Recall_own_rec_12 :  0.18249524913668436


In [16]:
print('MAP_own_rec_12 : ', result_test.apply(lambda row: ap_k(row['top_12_recs'], row['actual_article_id_short'], 12), axis=1).mean())

MAP_own_rec_12 :  0.029444920226172573


### Предсказания моделей LGBM и их оценка

In [43]:
# преобразуем тип данных поля age для модели LGBM
# result['age'] = result.apply(lambda row: col_convert_int(row['age']), axis=1)

In [44]:
%%time
customers_id, article_id, target, preds = get_lvl2_model_validation_LGBM_preds(result, features, cat_feats)

Получаем предсказания для возрастной группы 16-24
Получаем предсказания для возрастной группы 25-44
Получаем предсказания для возрастной группы 45-64
Получаем предсказания для возрастной группы 65-96
CPU times: user 46min 13s, sys: 52.5 s, total: 47min 6s
Wall time: 7min 29s


In [45]:
%%time
# построение датафрейма для получения оценок предсказаний модели
filename = 'validation_result'
result_test = get_preds_result(customers_id, article_id, target, preds, filename)

CPU times: user 24 s, sys: 690 ms, total: 24.7 s
Wall time: 28.1 s


In [46]:
print('Recall_own_rec_12 : ', result_test.apply(lambda row: recall(row['top_12_recs'], row['actual_article_id_short']), axis=1).mean())

Recall_own_rec_12 :  0.18500239150603634


In [47]:
print('MAP_own_rec_12 : ', result_test.apply(lambda row: ap_k(row['top_12_recs'], row['actual_article_id_short'], 12), axis=1).mean())

MAP_own_rec_12 :  0.029709670353044688
